# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [37]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com/"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = "status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}/{status_url}")
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(req.status_code)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [38]:
# Set the countries_url variable (1 line)
countries_url = "/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}/{countries_url}")
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)

403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [39]:
# Set the cookie_url variable (1 line)
cookie_url = "/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
req = requests.get(f"{root_url}/{cookie_url}")
cookies = req.json()
print(req.status_code, cookies)

200 {'message': 'The cookie has been created'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [40]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
req = requests.get(f"{root_url}/{countries_url}")
# display the countries variable (1 line)
print(countries)

{'message': 'The cookie is missing'}


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [41]:
# Set the leaders_url variable (1 line)
leaders_url = "/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(f"{root_url}/{leaders_url}")
# display the leaders variable (1 line)
print(leaders)

<Response [403]>


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [42]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
endpoint_cookie = "/cookies"
req_cookie = requests.get(f"{root_url}/{endpoint_cookie}")
cookie = req_cookie.cookies
print(req_cookie.status_code, cookie)

params = {
    'country': 'us'
}
# assign the output to the leaders variable (1 line)
leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookie, params=params)
# display the leaders variable (1 line)
leaders.json()

404 <RequestsCookieJar[]>


{'message': 'The cookie is missing'}

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [43]:
# 4 lines
leaders_per_country = {}

for country_code in countries:

    params = {'country': country_code}

    req_leaders = requests.get(f"{root_url}/{leaders_url}", params=params, cookies=cookie)

    if req_leaders.status_code == 200:
        leaders_per_country[country_code] = req_leaders.json()
    else:
        print(req_leaders.status_code, f"Failed to retrieve leaders from: {country_code}")

print(leaders_per_country)

403 Failed to retrieve leaders from: message
{}


In [44]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [45]:
# < 15 lines
def get_leaders():

    root_url = "https://country-leaders.onrender.com/"
    countries_url = "/countries"
    leaders_url = "/leaders"
    cookie_url = "/cookie"

    #Get cookies
    req_cookies = requests.get(f"{root_url}/{cookie_url}")
    cookie = req_cookies.cookies
    print(req_cookies.status_code, cookie)

    # Get countries list: ['us', 'be', 'ma', 'ru', 'fr']
    req_countries = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
    countries = req_countries.json()
    print(req_countries.status_code, countries)

    # Get leaders from each country and add them to dictionary.
    leaders_per_country = {}

    for country_code in countries:

        params = {'country': country_code}

        req_leaders = requests.get(f"{root_url}/{leaders_url}", params=params, cookies=cookie)

        if req_leaders.status_code == 200:
            leaders_per_country[country_code] = req_leaders.json()
        else:
            print(req_leaders.status_code, f"Failed to retrieve leaders from: {country_code}")

    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [46]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

200 <RequestsCookieJar[<Cookie user_cookie=c9b09189-bb8e-4c78-bc88-ff3f06eedffb for country-leaders.onrender.com/>]>
200 ['us', 'be', 'ma', 'ru', 'fr']
{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [47]:
# 3 lines

# Get the url from one leader
for country_code, leaders in leaders_per_country.items():
    for leader in leaders:
        if leader['id'] == 'Q169502':
            leader_wiki_url = leader['wikipedia_url']
            leader_first_name = leader['first_name']
            break
print(leader_wiki_url)
print(leader_first_name)

# Get raw html from url
req_leader_wiki_url = requests.get(leader_wiki_url)
print(req_leader_wiki_url.text)

https://fr.wikipedia.org/wiki/%C3%89mile_Loubet
Émile
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="fr" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Émile Loubet — Wikipédia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [48]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Émile Loubet — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clien

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [49]:
# 2 lines
paragraphs = [p.text for p in soup.find_all("p")]
print(paragraphs)

['Pour les articles homonymes, voir Loubet.\n', '\n', "Émile Loubet, né le 30 décembre 1838 à Marsanne (Drôme) et mort le 20 décembre 1929 à Montélimar (Drôme), est un homme d'État français. Il est président de la République française du 18 février 1899 au 18 février 1906.\n", "Avocat de profession, il est élu député de la Drôme en 1876. Après avoir été ministre des Travaux publics, il est président du Conseil de février à décembre 1892. Il est en parallèle ministre de l'Intérieur, fonction qu'il conserve dans le premier gouvernement Ribot. En 1896, il accède à la présidence du Sénat.\n", "En 1899, seul candidat en lice, il est élu président de la République après la mort soudaine de Félix Faure. Son mandat est notamment marqué par la fin de l'affaire Dreyfus, par l’affaire des fiches, et par le vote de la loi de séparation des Églises et de l'État à l'initiative d’Aristide Briand. À l'issue de son septennat, il se retire de la vie politique dans la Drôme.\n", "Né le 30 décembre 1838[N

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [50]:
# <10 lines
import string

first_paragraph = None

for p in paragraphs:
    if p.strip():
        first_word = p.lstrip().split()[0].strip(string.punctuation).lower()
        
        #print(f"Leader's first name: {leader['first_name'].lower()}")

        #print(f"Checking paragraph: {p}")
        #print(f"First word: {first_word}")

        if first_word == leader_first_name.lower():
            first_paragraph = p
            break
      
print(first_paragraph)

Émile Loubet, né le 30 décembre 1838 à Marsanne (Drôme) et mort le 20 décembre 1929 à Montélimar (Drôme), est un homme d'État français. Il est président de la République française du 18 février 1899 au 18 février 1906.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [51]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    leader_first_name = None

    for country_code, leaders in leaders_per_country.items():
        for leader in leaders:
            if leader['wikipedia_url'] == wikipedia_url:
                leader_first_name = leader['first_name']
                break
        if leader_first_name:
                break
        
    if not leader_first_name:
         print("Leader not found for this URL.")
         return None

    print(wikipedia_url)

    req_leader_wiki_url = requests.get(wikipedia_url)
    soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')

    paragraphs = [p.text for p in soup.find_all("p")]

    for first_paragraph in paragraphs:
        if first_paragraph.strip():
            first_word = first_paragraph.lstrip().split()[0].strip(string.punctuation).lower()
            if first_word == leader_first_name.lower():
                return first_paragraph
            
    return None

In [52]:
# Test: 3 lines
print(get_first_paragraph('https://fr.wikipedia.org/wiki/Emmanuel_Macron'))
print(get_first_paragraph('https://en.wikipedia.org/wiki/George_Washington'))
# Problem because dictionary had English name while the url is in Cyrillic
print(get_first_paragraph('https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87'))

https://fr.wikipedia.org/wiki/Emmanuel_Macron
Emmanuel Macron ([ɛmanɥɛl makʁɔ̃][e] Écouter), né le 21 décembre 1977 à Amiens (Somme), est un haut fonctionnaire, banquier d'affaires et homme d'État français. Il est Président de la République française depuis le 14 mai 2017.

https://en.wikipedia.org/wiki/George_Washington
George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".

https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [70]:
# 3 lines
import re

#def is_non_latin(text):
#    return bool(re.search(r'[^\x00-\x7F]', text))

def is_arabic(text):
    return bool(re.search(r'[\u0600-\u06FF]', text))

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines
"""
def get_first_paragraph(wikipedia_url):
    leader_first_name = None

    for country_code, leaders in leaders_per_country.items():
        for leader in leaders:
            if leader['wikipedia_url'] == wikipedia_url:
                leader_first_name = leader['first_name']
                break
        if leader_first_name:
                break
        
    if not leader_first_name:
         print("Leader not found for this URL.")
         return None

    print(wikipedia_url)

    req_leader_wiki_url = requests.get(wikipedia_url)
    soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')

    paragraphs = soup.find_all("p")

    if is_non_latin(req_leader_wiki_url.text):
         for p in paragraphs:
              if p.find("b"):
                   return p.text.strip()
    else:
         for first_paragraph in paragraphs:
            if first_paragraph.text.strip():
                first_word = first_paragraph.text.lstrip().split()[0].strip(string.punctuation).lower()
                if first_word == leader_first_name.lower():
                    return first_paragraph.text.strip()

    
            
    return None
"""

def get_first_paragraph(wikipedia_url):
    req_leader_wiki_url = requests.get(wikipedia_url)
    soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')
    paragraphs = soup.find_all("p")

    for p in paragraphs:
        if p.find("b"):
            first_paragraph = p.text.strip()
            # Removes citations e.g. [4]
            first_paragraph_cleaned = re.sub(r'\[\d+]', '', first_paragraph).strip()
            
            #checks if arabic script and flips the paragraph around.
            if is_arabic(first_paragraph_cleaned):
                first_paragraph_cleaned = first_paragraph_cleaned[::-1]

            return first_paragraph_cleaned
    
    return None

print(get_first_paragraph('https://en.wikipedia.org/wiki/George_Washington'))
print(get_first_paragraph('https://fr.wikipedia.org/wiki/Jean_Casimir-Perier'))
print(get_first_paragraph('https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87'))
print(get_first_paragraph('https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81'))


George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Jean Casimir-Perier, né le 8 novembre 1847 à Paris et mort le 11 mars 1907 dans la même ville, est un homme d'État français. Il est président de la République française du 27 juin 1894 au 16 janvier 1895.
Влади́мир Влади́мирович Пу́тин (род. 7 октября 1952, Ленинград, СССР) — российский государственный и политический деятель. Действующий президент Российской Федерации, председатель Государственного Совета Российской Фед

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [55]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [73]:
# < 20 lines
import requests
from bs4 import BeautifulSoup
import re

def get_leaders():

    root_url = "https://country-leaders.onrender.com/"
    countries_url = "/countries"
    leaders_url = "/leaders"
    cookie_url = "/cookie"

    #Get cookies
    req_cookies = requests.get(f"{root_url}/{cookie_url}")
    cookie = req_cookies.cookies

    # Get countries list: ['us', 'be', 'ma', 'ru', 'fr']
    req_countries = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
    countries = req_countries.json()

    # Get leaders from each country and add them to dictionary.
    leaders_per_country = {}

    for country_code in countries:

        params = {'country': country_code}

        req_leaders = requests.get(f"{root_url}/{leaders_url}", params=params, cookies=cookie)

        if req_leaders.status_code == 200:
            leaders_per_country[country_code] = req_leaders.json()

            for leader in leaders_per_country[country_code]:
                wikipedia_url = leader.get("wikipedia_url")
                if wikipedia_url:
                    first_paragraph = get_first_paragraph(wikipedia_url)
                    leader["first_paragraph"] = first_paragraph

        else:
            print(req_leaders.status_code, f"Failed to retrieve leaders from: {country_code}")

    return leaders_per_country


def is_arabic(text):
    return bool(re.search(r'[\u0600-\u06FF]', text))


def get_first_paragraph(wikipedia_url):
    req_leader_wiki_url = requests.get(wikipedia_url)
    soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')
    paragraphs = soup.find_all("p")

    for p in paragraphs:
        if p.find("b"):
            first_paragraph = p.text.strip()
            # Removes citations e.g. [4]
            first_paragraph_cleaned = re.sub(r'\[\d+]', '', first_paragraph).strip()
            
            #checks if arabic script and flips the paragraph around.
            if is_arabic(first_paragraph_cleaned):
                first_paragraph_cleaned = first_paragraph_cleaned[::-1]

            return first_paragraph_cleaned
    
    return None


In [74]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
print(leaders_per_country)

403 Failed to retrieve leaders from: be
403 Failed to retrieve leaders from: ma
403 Failed to retrieve leaders from: ru
403 Failed to retrieve leaders from: fr
{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has t

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [2]:
# < 25 lines
import requests
from bs4 import BeautifulSoup
import re

def get_leaders():

    root_url = "https://country-leaders.onrender.com/"
    countries_url = "/countries"
    leaders_url = "/leaders"
    cookie_url = "/cookie"

    #Get cookies
    req_cookies = requests.get(f"{root_url}/{cookie_url}")
    cookie = req_cookies.cookies

    # Get countries list: ['us', 'be', 'ma', 'ru', 'fr']
    req_countries = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
    countries = req_countries.json()

    # Get leaders from each country and add them to dictionary.
    leaders_per_country = {}

    for country_code in countries:

        params = {'country': country_code}
        
        try:

            req_leaders = requests.get(f"{root_url}/{leaders_url}", params=params, cookies=cookie)
            
            if req_leaders.status_code == 403:
                req_cookies = requests.get(f"{root_url}/{cookie_url}")
                cookie = req_cookies.cookies
                req_leaders = requests.get(f"{root_url}/{leaders_url}", params=params, cookies=cookie)

            
            if req_leaders.status_code == 200:
                leaders_per_country[country_code] = req_leaders.json()

                for leader in leaders_per_country[country_code]:
                    wikipedia_url = leader.get("wikipedia_url")
                    if wikipedia_url:
                        first_paragraph = get_first_paragraph(wikipedia_url)
                        if first_paragraph:
                            print(f"{first_paragraph}")

            else:
                print(req_leaders.status_code, f"Failed to retrieve leaders from: {country_code}")

        except requests.RequestException as e:
            print(f"Could not retreive leaders for {country_code}: {e}")



def is_arabic(text):
    return bool(re.search(r'[\u0600-\u06FF]', text))


def get_first_paragraph(wikipedia_url):
    req_leader_wiki_url = requests.get(wikipedia_url)
    soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')
    paragraphs = soup.find_all("p")

    for p in paragraphs:
        if p.find("b"):
            first_paragraph = p.text.strip()
            # Removes citations e.g. [4], symbols and phonetic guides
            first_paragraph_cleaned = re.sub(r'\[\d+\]|\[.*?\]|/.*?/|[ɪˈˌːθðʃʒŋʊəɛɔɑɒʔʲʷ̃ɡʧʤ]|[ⓝⓘⓒⓗⓖⓞⓟⓑⓘⓛⓢ]|;|\b[a-z]+-[A-Z]+\b|\b[A-Z]+-[a-z]+\b', '', first_paragraph).strip()

            #checks if arabic script and flips the paragraph around.
            if is_arabic(first_paragraph_cleaned):
                first_paragraph_cleaned = first_paragraph_cleaned[::-1]

            return first_paragraph_cleaned
    
    return None


Check the output of your function again.

In [3]:
# Check the output of your function (1 line)
get_leaders()

George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.
Abraham Lincoln ( LINK-n February 12, 1809 – A

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [4]:
# < 20 lines
def get_first_paragraph(wikipedia_url, session):
    req_leader_wiki_url = session.get(wikipedia_url)
    soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')
    paragraphs = soup.find_all("p")

    for p in paragraphs:
        if p.find("b"):
            first_paragraph = p.text.strip()
            # Removes citations e.g. [4], symbols and phonetic guides
            first_paragraph_cleaned = re.sub(r'\[\d+\]|\[.*?\]|/.*?/|[ɪˈˌːθðʃʒŋʊəɛɔɑɒʔʲʷ̃ɡʧʤ]|[ⓝⓘⓒⓗⓖⓞⓟⓑⓘⓛⓢ]|;|\b[a-z]+-[A-Z]+\b|\b[A-Z]+-[a-z]+\b', '', first_paragraph).strip()

            #checks if arabic script and flips the paragraph around.
            if is_arabic(first_paragraph_cleaned):
                first_paragraph_cleaned = first_paragraph_cleaned[::-1]

            return first_paragraph_cleaned
    
    return None

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [9]:
def get_leaders():

    root_url = "https://country-leaders.onrender.com/"
    countries_url = "/countries"
    leaders_url = "/leaders"
    cookie_url = "/cookie"

    with requests.Session() as session:
        session.get(f"{root_url}/{cookie_url}")

        #Get cookies
        #req_cookies = requests.get(f"{root_url}/{cookie_url}")
        #cookie = req_cookies.cookies

        # Get countries list: ['us', 'be', 'ma', 'ru', 'fr']
        req_countries = session.get(f"{root_url}/{countries_url}")
        countries = req_countries.json()

        # Get leaders from each country and add them to dictionary.
        leaders_per_country = {}

        for country_code in countries:

            params = {'country': country_code}
        
            try:

                req_leaders = session.get(f"{root_url}/{leaders_url}", params=params)
            
                if req_leaders.status_code == 403:
                    session.get(f"{root_url}/{cookie_url}")
                    req_leaders = session.get(f"{root_url}/{leaders_url}", params=params)

            
                if req_leaders.status_code == 200:
                    leaders_per_country[country_code] = req_leaders.json()

                    for leader in leaders_per_country[country_code]:
                        wikipedia_url = leader.get("wikipedia_url")
                        if wikipedia_url:
                            first_paragraph = get_first_paragraph(session, wikipedia_url)
                            if first_paragraph:
                                print(f"{first_paragraph}")

                else:
                    print(req_leaders.status_code, f"Failed to retrieve leaders from: {country_code}")

            except requests.RequestException as e:
                print(f"Could not retreive leaders for {country_code}: {e}")

def is_arabic(text):
    return bool(re.search(r'[\u0600-\u06FF]', text))

def get_first_paragraph(session, wikipedia_url):
    req_leader_wiki_url = session.get(wikipedia_url)
    soup = BeautifulSoup(req_leader_wiki_url.content, 'html.parser')
    paragraphs = soup.find_all("p")

    for p in paragraphs:
        if p.find("b"):
            first_paragraph = p.text.strip()
            # Removes citations e.g. [4], symbols and phonetic guides
            first_paragraph_cleaned = re.sub(r'\[\d+\]|\[.*?\]|/.*?/|[ɪˈˌːθðʃʒŋʊəɛɔɑɒʔʲʷ̃ɡʧʤ]|[ⓝⓘⓒⓗⓖⓞⓟⓑⓘⓛⓢ]|;|\b[a-z]+-[A-Z]+\b|\b[A-Z]+-[a-z]+\b', '', first_paragraph).strip()

            #checks if arabic script and flips the paragraph around.
            if is_arabic(first_paragraph_cleaned):
                first_paragraph_cleaned = first_paragraph_cleaned[::-1]

            return first_paragraph_cleaned
    
    return None

Test your new functions.



In [8]:
get_leaders()

# Small improvement: Shaved time down to 1m 6.9s from 1m 12.2s

George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.
Abraham Lincoln ( LINK-n February 12, 1809 – A

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [62]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [63]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [64]:
# 3 lines


In [65]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!